In [12]:
print("ok")

ok


In [13]:
%pwd

'c:\\Users\\jalandhar\\Desktop\\CollegeProject\\MediChat\\research'

In [14]:
import os
os.chdir("../")

In [64]:
%pwd

'c:\\Users\\jalandhar\\Desktop\\CollegeProject\\MediChat'

In [65]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI


In [17]:
#Extract Data From the PDF File
def load_pdf_file(data):
        loader= DirectoryLoader(data,
                                glob="*.pdf",
                                loader_cls=PyPDFLoader)
        documents=loader.load()
        return documents

In [18]:
extracted_data=load_pdf_file(data='KnowledgeBase/')

In [ ]:
# extracted_data

In [66]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [57]:
# chunk = text_chunks[0]
# vector = embeddings.embed_query(chunk.page_content)
# print(len(vector))


In [ ]:
# for i, chunk in enumerate(text_chunks):
#     print(f"Chunk {i} length: {len(chunk.page_content)}")


In [67]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))


Length of Text Chunks 5860


In [75]:
from langchain.embeddings import HuggingFaceEmbeddings

In [76]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [77]:
embeddings = download_hugging_face_embeddings()

C:\Users\jalandhar\AppData\Local\Temp\ipykernel_14460\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [78]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [80]:
#query_result

In [81]:
from dotenv import load_dotenv
load_dotenv()

True

In [82]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY=os.environ.get('GEMINI_API_KEY')

In [83]:
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medichat"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medichat",
    "metric": "cosine",
    "host": "medichat-bst0i7a.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [84]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [85]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [86]:
# Load Existing index
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [87]:
docsearch

In [88]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [89]:
retrieved_docs = retriever.invoke("What is Acne?")

In [90]:
retrieved_docs

[Document(id='dffd7d69-2b97-444e-9675-0ae8687fd326', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'KnowledgeBase\\Gale_Encyclopedia_of_Medicine.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='e1dc1f35-8fb6-4a78-b409-02f0ebabb506', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'KnowledgeBase\\Gale_Encyclopedia_of_Medicine.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo

In [91]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.4, max_output_tokens=500, google_api_key=GEMINI_API_KEY)

In [107]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are a highly-concise, factual assistant.  "
    "If the user message is just a greeting (e.g. “hi”, “hello”), reply with a brief friendly greeting.  "
    "If the user offers an apology (e.g. “sorry”), respond with a polite acknowledgement and offer help.  "
    "Otherwise, read the CONTEXT carefully and answer the QUESTION in no more than three sentences.  "
    "Base your response strictly on the CONTEXT—do not hallucinate.  "
    "If the CONTEXT does not contain the answer, say “I don’t know.”\n\n"
    "CONTEXT:\n{context}\n\n"
)

prompt = ChatPromptTemplate.from_messages(
                         [
                             ("system", system_prompt),
                             ("human", "{input}"),     
                         ] 
)

In [108]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [109]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Acne vulgaris is the medical term for common acne and is the most common skin disease.


In [110]:
response = rag_chain.invoke({"input": "tell me fever medicine"})
print(response["answer"])

Aspirin and acetaminophen (Tylenol) can reduce fever. Aspirin lowers fever by acting on the part of the brain that regulates temperature. Acetaminophen may relieve fever over 101°F (38.3°C).


In [111]:
response = rag_chain.invoke({"input": "what is agentic ai?"})
print(response["answer"])

I am sorry, but the context provided does not contain information about agentic AI. Can I help with something else?
